In [1]:
import sys
import os

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    precision_score,
    recall_score,
    classification_report,
)


# Local module imports
sys.path.insert(0, "..") # adds parent folder to sys path
#sys.path.insert(0, "../datasets")

#from datasets.data_loading import preprocess_amazon_data

In [2]:
def preprocess_pandas(data, columns):
    df_ = pd.DataFrame(columns=columns)
    data["Sentence"] = data["Sentence"].str.lower()
    data["Sentence"] = data["Sentence"].replace(
        "[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+", "", regex=True
    )  # remove emails
    data["Sentence"] = data["Sentence"].replace(
        "((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}", "", regex=True
    )  # remove IP address
    data["Sentence"] = data["Sentence"].str.replace(
        "[^\w\s]", ""
    )  # remove special characters
    data["Sentence"] = data["Sentence"].replace("\d", "", regex=True)  # remove numbers
    for index, row in data.iterrows():
        word_tokens = word_tokenize(row["Sentence"])
        filtered_sent = [w for w in word_tokens if not w in stopwords.words("english")]
        df_ = df_.append(
            {
                "index": row["index"],
                "Class": row["Class"],
                "Sentence": " ".join(filtered_sent[0:]),
            },
            ignore_index=True,
        )
    return data

In [3]:
DATASET_PATH = "../datasets/amazon_cells_labelled.txt"

# get data, pre-process and split
data = pd.read_csv(DATASET_PATH, delimiter="\t", header=None)
data.columns = ["Sentence", "Class"]
data["index"] = data.index  # add new column index
columns = ["index", "Class", "Sentence"]
data = preprocess_pandas(data, columns)  # pre-process
(training_data, validation_data, training_labels, validation_labels) = train_test_split(  # split the data into training, validation, and test splits
    data["Sentence"].values.astype("U"),
    data["Class"].values.astype("int32"),
    test_size=0.10,
    random_state=0,
    shuffle=True,
)

# vectorize data using TFIDF and transform for PyTorch for scalability
word_vectorizer = TfidfVectorizer(
    analyzer="word",
    ngram_range=(1, 2),
    max_features=50000,
    max_df=0.5,
    use_idf=True,
    norm="l2",
)
training_data = word_vectorizer.fit_transform(training_data)  # transform texts to sparse matrix
training_data = training_data.todense()  # convert to dense matrix for Pytorch
vocab_size = len(word_vectorizer.vocabulary_)
validation_data = word_vectorizer.transform(validation_data)
validation_data = validation_data.todense()
train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [4]:
# Combine data with labels
train_dataset = TensorDataset(train_x_tensor, train_y_tensor)
validation_dataset = TensorDataset(validation_x_tensor, validation_y_tensor)

trainloader = DataLoader(train_dataset, batch_size=len(training_data), shuffle=True)
validationloader = DataLoader(validation_dataset, batch_size=len(validation_data), shuffle=False)

In [5]:
network = nn.Sequential(
    nn.Linear(vocab_size, 100),
    nn.ReLU(), 
    nn.Linear(100, 2)
)

optimizer = torch.optim.Adam(network.parameters(), lr=0.01)
loss_function = nn.CrossEntropyLoss()

epochs = 1

for epoch in range(epochs):
    for batch_nr, (data, labels) in enumerate(trainloader):
        prediction = network(data)
        
        loss = loss_function(prediction, labels)
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        #Print the epoch, batch, and loss
        print(
            f'\rEpoch {epoch+1} [{batch_nr+1}/{len(trainloader)}] - Loss: {loss}',
            end=''
        )

Epoch 1 [1/1] - Loss: 0.6932216286659241

In [6]:
with torch.no_grad():
    correct = 0
    total = 0
    # For each batch of testing data (since the dataset is too large to run all data through the network at once)
    for batch_nr, (data, labels) in enumerate(validationloader):
        prediction = network(data)
        
        # Get the predicted category from each prediction
        prediction = list(pred.argmax() for pred in prediction)
        
        # Sum all the predictions that were the same as the label
        correct += np.equal(prediction, labels).sum().item()
        total += len(labels)
    
    print(f'The accuracy of the network is {str(100*correct/total)[:4]}%.')

The accuracy of the network is 78.0%.


In [7]:
# Let's print a sentence and predict it's category
sentence_index = 0
#prediction = torch.argmax(network(torch.unsqueeze(bow_embedder(ag_train[sentence_index][1], len(vocab)), dim=0))).item()
#print(
#    f'The network predicted that \n"{tensor2text(ag_train[sentence_index][1])}"\n should be in the category {ag_labels[prediction]}'
#)

sentence = "this product sucks"
sentence_label = 0

